# Иерархическая кластеризация


Используем реализацию [иерархической кластеризации](https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering) в [scikit-learn](https://scikit-learn.org/stable/index.html) (метод [AgglomerativeClustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html))

In [ ]:
import pandas as pd
import numpy as np

from sklearn import preprocessing  # методы для предварительной обработки данных (нормирование etc)
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics

from scipy.cluster.hierarchy import linkage, dendrogram # Построение дендрограммы

import seaborn as sns # 2D-визуалиазция
import plotly.express as px # 3D-визуалиазация
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('Countries.csv', sep=';', decimal=',')
data.head()

In [ ]:
countries = list(data['Страны'])

In [ ]:
# Удалим нечисловые переменные
data = data.drop(columns=['Страны'])
data

## Нормирование данных

In [ ]:
data_norm = preprocessing.StandardScaler(with_mean=True, with_std=True).fit_transform(data)
# data_norm = preprocessing.MinMaxScaler().fit_transform(data)
# data_norm = preprocessing.RobustScaler().fit_transform(data)

## Дендрограмма
Дендрограмма существенно зависит от выбора расстояния.

Нужно выбирать тот метод, который приведет к хорошо интерпретируемым кластерам

In [ ]:
mergings = linkage(data_norm, method='ward')  # попробуем прогнать несколько раз, меняя method

In [ ]:
plt.figure(figsize=(25, 25))  # размер картинки

dendrogram(mergings,
           labels=countries,
           leaf_rotation=90,
           leaf_font_size=20,
           )

plt.show()

## Кластеризация

In [ ]:
# Специфицируем процедуру
cluster = AgglomerativeClustering(n_clusters=3, metric='euclidean', 
                                  linkage='ward', compute_full_tree=True, distance_threshold=None)
# проводим кластеризацию
cl = cluster.fit_predict(data_norm)

print(cl)

## Метрики кластеризации
[Основные метрики](https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation)
| Метрика |Метод |Критерий|
|-|-|-|
|Silhouette Coefficient|silhouette_score| max|
|Calinski-Harabasz|calinski_harabasz_score|max|
|Davies-Bouldin|davies_bouldin_score|min|

In [ ]:
print('Silhouette Score=', metrics.silhouette_score(data_norm, cl))
print('Calinski-Harabasz Score=', metrics.calinski_harabasz_score(data_norm, cl))
print('Davies-Bouldin Score=', metrics.davies_bouldin_score(data_norm, cl))

## Визуализация
используем библиотеки [seaborn](https://seaborn.pydata.org/index.html) и [plotly](https://plotly.com/python/)

In [ ]:
data['Cluster'] = cl
data.head()

In [ ]:
sns.scatterplot(data=data, x='ИРЧП', y='Население', hue='Cluster')

In [ ]:
sns.scatterplot(data=data, x='ИРЧП', y='ВВП д/н', hue='Cluster')

In [ ]:
fig = px.scatter_3d(data, x='ИРЧП', y='ВВП д/н', z='Безработица (%)', color='Cluster')
fig.update_traces(marker_size=3)
fig.show()